In [1]:
import pandas as pd
import numpy as np
import math
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Functions

In [26]:
def get_shot_factors(input_dict, reg_ball_speed, reg_launch_angle, reg_azimuth, reg_side_spin, reg_back_spin):
    
    cols = ['width_coeff', 'x_coeff', 'y_coeff', 'flight_length', 'width_half_coeff',
            'x_half_coeff', 'y_half_coeff', 'width_sc', 'x_sc', 'y_sc']
    mod_input = {col:[input_dict[col]] for col in cols}
    df = pd.DataFrame(mod_input)
    df.columns = cols
    
    df2 = df.copy()
    df2['x_y_coeff'] = np.sqrt(df2['x_coeff']**2+df2['y_coeff']**2)

    df2['x_y_half_coeff'] = np.sqrt(df2['x_half_coeff']**2+df2['y_half_coeff']**2)

    cols = [c for c in df2.columns if c not in ["shot_no","ball_speed","launch_angle",
                                                "azimuth","side_spin","back_spin"]]

    col = 'flight_length'
    i = cols.index(col)
    rest = cols[:i]+cols[i+1:]
    for r_col in rest:
            df2[col+"_"+r_col] = df2[col]*df2[r_col]
    
    df2 = df2[['x_coeff', 'flight_length', 'x_half_coeff', 'width_sc', 'y_sc',
               'x_y_coeff', 'flight_length_x_coeff', 'flight_length_y_coeff',
               'flight_length_y_half_coeff', 'flight_length_y_sc',
               'flight_length_x_y_coeff', 'flight_length_x_y_half_coeff']]
    
    return {"ball_speed" : reg_ball_speed.predict(df)[0],
            "launch_angle" : reg_launch_angle.predict(df)[0],
            "azimuth" : reg_azimuth.predict(df)[0],
            "side_spin" : reg_side_spin.predict(df2)[0],
            "back_spin" : reg_back_spin.predict(df)[0]}

# Retreiving models

In [27]:
with open('../models/reg_ball_speed.pkl', 'rb') as f:
    reg_ball_speed = pickle.load(f)

with open('../models/reg_launch_angle.pkl', 'rb') as f:
    reg_launch_angle = pickle.load(f)

with open('../models/reg_azimuth.pkl', 'rb') as f:
    reg_azimuth = pickle.load(f)
    
with open('../models/reg_back_spin.pkl', 'rb') as f:
    reg_back_spin = pickle.load(f)
    
with open('../models/reg_side_spin.pkl', 'rb') as f:
    reg_side_spin = pickle.load(f)

# Sample Input

In [28]:
data = pd.read_csv("shot_factors_features.csv")

In [29]:
data2 = data[[c for c in data.columns if c not in ["shot_no","ball_speed","launch_angle",
                                               "azimuth","side_spin","back_spin"]]]

In [30]:
data_point = data2.iloc[0].to_dict()

In [31]:
data_point

{'width_coeff': -0.0553412766754512,
 'x_coeff': -0.01298389133389271,
 'y_coeff': -0.07035135525842252,
 'flight_length': 10.0,
 'width_half_coeff': -0.0891270363562683,
 'x_half_coeff': -0.010350477173429516,
 'y_half_coeff': -0.08079645046603966,
 'width_sc': 0.21421487552599025,
 'x_sc': 0.7705521932710807,
 'y_sc': -0.3209023197228837}

# Prediction

In [32]:
shot_factors = get_shot_factors(data_point, reg_ball_speed, reg_launch_angle,
                                reg_azimuth, reg_side_spin, reg_back_spin)

In [33]:
shot_factors

{'ball_speed': 79.69344911145016,
 'launch_angle': 37.76465045992494,
 'azimuth': -6.721174521721162,
 'side_spin': -781.1045876745029,
 'back_spin': 5784.022628852465}

In [34]:
data.iloc[0,:][["ball_speed","launch_angle","azimuth","side_spin","back_spin"]]

ball_speed        78.3
launch_angle      38.3
azimuth           -6.5
side_spin       -208.0
back_spin       6631.0
Name: 0, dtype: float64